<a href="https://colab.research.google.com/github/HungYangChang/ECSE551/blob/master/Miniproject1_james.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fetch Hepatitis Data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

url = "https://raw.githubusercontent.com/jonarsenault/ecse551data/master/hepatitis.csv"
# url = "https://raw.githubusercontent.com/jonarsenault/ecse551data/master/bankrupcy.csv"
data = pd.read_csv(url)

# # check if data was read successfully or not
print(data.head())

# data shape
data.shape

# Plot
Task: what are the distribution of some of the features? etc. You may visualize your results using histogram plots.

Plot the distribution of data



In [ ]:
plt.subplot(1,2,1)
sns.distplot(data['age'])
plt.subplot(1,2,2)
sns.distplot(data['sex'])

In [ ]:
plt.subplot(1,2,1)
sns.distplot(data['steroid'])
plt.subplot(1,2,2)
sns.distplot(data['antivirals'])

In [ ]:
plt.subplot(1,2,1)
sns.distplot(data['fatigue'])
plt.subplot(1,2,2)
sns.distplot(data['malaise'])

In [ ]:
plt.subplot(1,2,1)
sns.distplot(data['anorexia'])
plt.subplot(1,2,2)
sns.distplot(data['liver_big'])

In [ ]:
plt.subplot(1,2,1)
sns.distplot(data['liver_firm'])
plt.subplot(1,2,2)
sns.distplot(data['spleen_palable'])

In [ ]:
plt.subplot(1,2,1)
sns.distplot(data['spiders'])
plt.subplot(1,2,2)
sns.distplot(data['ascites'])

In [ ]:
plt.subplot(1,2,1)
sns.distplot(data['varices'])
plt.subplot(1,2,2)
sns.distplot(data['bilirubin'])

In [ ]:
plt.subplot(1,2,1)
sns.distplot(data['alk_phosphate'])
plt.subplot(1,2,2)
sns.distplot(data['sgot'])

In [ ]:
plt.subplot(1,3,1)
sns.distplot(data['albumin'])
plt.subplot(1,3,2)
sns.distplot(data['protime'])
plt.subplot(1,3,3)
sns.distplot(data['histology'])

# Analysize Data
Task: Perform some statistical analysis on the datasets e.g. what are the distribution of the two classes? 

Find mean, std, correlation (what others?...)

In [ ]:
# Analysize Data
# Find mean and std
data.iloc[:,:-1].describe()
n_colum = int(data.size/len(data))

# Find correlation of each one (to check whether i.i.d assumpation is correct enough or not)
corr_data = data.corr()
plt.figure(figsize=(n_colum, n_colum))
heatmap = sns.heatmap(corr_data, annot=True, linewidths=0.2, linecolor="white",vmin=0.3, cmap="PuRd")

# Split Data: Training and Test Data
split oriningal data to training data and Test data



In [ ]:
# split data into Training, validation, and Test (Q: is validation necessary?)
### k-fold non-finished yet ###

# 80% for training

def splitdata(data,perc_training):
    data_train = data.sample(frac = perc_training, random_state=1)
    ##.iloc to get row information but not include label(last one)
    x_train = np.array(data_train.iloc[:, :-1])
    print (x_train.shape)
    y_train = np.array(data_train["ClassLabel"])
    print (y_train.shape)

    ## Select rest of data for testing
    data_test = data.drop(data_train.index)
    x_test = np.array(data_test.iloc[:, :-1])
    y_test = np.array(data_test["ClassLabel"])
    print("Number of Training data: ", len(x_train),"Testing data:",len(x_test))
    return x_train, y_train, x_test, y_test
  

# linear classifier logistic regression

Implementing ***Linear classifier logistic regression***
The main aim of the Hepatitis dataset is to discriminate two classes: survivors and patients for whom the hepatitis proved terminal.

You are free to implement the method in anyway you want, however we recommend to implement both models as python classes (use of constructor is recommended). Each of your model class should have at least two functions: *fit* and *predict*. The function *fit* takes the training data X and its corresponding labels vector y as well as other hyperparameters (such as learning rate) as input, and execute the model training through modifying the model parameters (i.e. W). *predict* takes a set of test data as input and outputs predicted labels for the input points. Note that you need to convert probabilities to binary 0-1 predictions by thresholding the output at 0.5. The ground-truth labels should also be converted to binary 0-1.

you might investigate different stopping criteria for the gradient descent in logistic regression or develop an automated approach to select a good subset of features.

linear classifier logistic regression
1. normalization
2. sigmoid function
3. fit
4. predict



In [ ]:
## linear classifier logistic regression

class Linear_logistic_classifier:

  ## Initializing weights by random
  # feed initial weight into function
  def __init__(self, weights):
      self.w = weights
      pass

  ## determine whether normalization (for training dta, norm = True)
  def normalization(self, x, norm = True):
      if norm:
        self.x_mean  = np.mean(x)
        self.x_std = np.std(x)
        return (x-self.x_mean)/(self.x_std)
      else:
        return x
  
  ## sigmoid function
  def sigmoid(self,x):
      return 1/(1+np.exp(-x))

  '''
  ## fit function
  lr: learning rate
  iter_time: total times for iteration (even if it's not converge)
  stop_criteria : If the loss is less than this value, then stop
  '''
  def fit(self, x_train, y_train, lr, iter_time, stop_criteria = 1e-2, norm = True, epsilon =1e-6): 
      x_train = self.normalization(x_train, norm)
      y_train = self.normalization(y_train, norm)
      CEstore = []
      DCEstore = []
      for iteration in range(iter_time):
          wtx = np.dot(self.w.T, x_train.T)
          # Cross-entropy(CE)
          CE = -1 * np.sum(
              np.sum(np.dot(y_train, np.log(self.sigmoid(wtx).T + epsilon),
              np.dot((1-y_train), np.log((1- self.sigmoid(wtx)).T + epsilon))))
              ).reshape(-1, 1)
          # Derivative of C
          DCE = -1* np.sum(
              np.dot(x_train.T,(y_train-self.sigmoid((wtx))).T)
              ).reshape(-1, 1)
          # weight update
          self.w -= lr*DCE
          CEstore.append(CE.flatten())
          DCEstore.append(DCE.flatten())

      ### TODO stop criteria
      return CEstore, DCEstore

  ## predict function : normalize data first, and then feed x_test 
  def predict(self, x_test, y_test, norm = True):
      x_test = self.normalization(x_test, norm)
      y_test = self.normalization(y_test, norm)
      
      y_predict_prob = self.sigmoid(np.dot(self.w.T,x_test.T))
      y_predict = np.where(y_predict_prob < 0.5,0,1)
      return y_predict

In [ ]:
          CE = -1 * np.sum(
              np.sum(np.dot(y_train, np.log(self.sigmoid(wtx).T + epsilon),
              np.dot((1-y_train), np.log((1- self.sigmoid(wtx)).T + epsilon))))
              ).reshape(-1, 1)
          # Derivative of C
          DCE = -1* np.sum(
              np.dot(x_train.T,(y_train-self.sigmoid((wtx))).T)
              ).reshape(-1, 1)

1. evlu_acc
2. initial weights

In [ ]:
def evalu_acc(y_test, y_predict):
    return np.mean(y_test == y_predict)

def initial_weights(x_train, init_constant=0.3):
    weights = np.array(init_constant*np.random.rand(x_train.shape[1],1))
    return weights

Run the classifier

In [ ]:
# split data
x_train, y_train, x_test, y_test = splitdata(data,0.8)

# generate weights
weights = initial_weights(x_train, 0.2)
print ("type:", weights.shape)

# Use Classifier
reg = Linear_logistic_classifier(weights)
CE, DCE = reg.fit(x_train, y_train, 0.02, 500, 0.01, True, 1e-6)
y_predict = reg.predict(x_test, y_test, True)
accuracy = evalu_acc(y_test, y_predict)
print ("the accuracy is", accuracy)
print (CE)

plt.title('CE function')
plt.xlabel('iterations'), plt.ylabel('CE function')
plt.plot(CE)
plt.show()

print (DCE)

plt.title('CE function')
plt.xlabel('iterations'), plt.ylabel('CE function')
plt.plot(DCE)
plt.show()


In [ ]:
# k-fold


In [ ]:
# Changing learning rate, check if it's overfit

# Report.pdf
•   Does the report include all the required experiments?
•  Is the report technically sound? (i.e. do the steps taken make sense? Are the results
in an acceptable range?)
•  How thorough/rigorous is the experimental validation?
•  Is the report well-organized and coherent?
•  Is the report clear and free of grammatical errors and typos?
•  Does the report contain sufficient and appropriate references and related work?
